In [1]:
# Use the following features to derive new features:
# lang (en, und, tt) ... convert to a bool for each language
# to_user_name -> bool for conversational tweet
# from_user_lang .. alternative to loang ()probably not needed)
# source -> contains "instagram.com" becomes is_instagram
# tweet_text_contains_url -> needs a regex (google) or just http?
# also need to filter out null LGA as they are useless to us
# then need to summarise the new features per LGA

# SEPARATE TASK : Improve smc_tokenizer punct, words ending in !

In [2]:
import numpy as np
import pandas as pd
import os
import re
base = "../../data/clean/"
pd.set_option('display.max_rows', 150)

In [3]:
tweetdf = pd.read_csv(base+'loctrisma2016.csv')
tweetdf = tweetdf.loc[~pd.isnull(tweetdf['lga'])].reset_index()
len(tweetdf)

/Users/richardnichol/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


807489

In [5]:
tweetdf.head()

,index,Unnamed: 0,tweet_id,created_at,from_user_id,source,text,to_user_id,to_user_name,in_reply_to_status_id,...,location_name,country,location_full_name,location_type,country_code,collected_at,lga,state,lat,lng
0,2,22,7.964761e+17,2016-11-09 22:14:37.000000,4.721718e+09,"<a href=""http://www.rfs.nsw.gov.au"" rel=""nofol...",ADVICE. Bush Fire. Dungog (Mt Ebsworth) at 10 ...,NaN,NaN,NaN,...,Cambra,Australia,"Cambra, New South Wales",neighborhood,AU,2017-05-14,Dungog,NSW,-32.442062,151.875297
1,3,26,7.964002e+17,2016-11-09 17:13:06.000000,4.721718e+09,"<a href=""http://www.rfs.nsw.gov.au"" rel=""nofol...",ADVICE. Bush Fire. Port Macquarie-Hastings (15...,NaN,NaN,NaN,...,Marlo Merrican,Australia,"Marlo Merrican, New South Wales",neighborhood,AU,2017-05-14,Port Macquarie-Hastings,NSW,-31.228983,152.671562
2,4,39,7.158478e+17,2016-04-01 10:26:37.000000,4.096223e+07,"<a href=""http://www.twitter.com"" rel=""nofollow...",About half way to Dubbo https://t.co/pslONhZKdY,NaN,NaN,NaN,...,Lithgow,Australia,"Lithgow, New South Wales",city,AU,2013-01-01 00:00:01,Lithgow,NSW,-33.482734,150.157017
3,5,43,7.740848e+17,2016-09-09 03:19:33.000000,4.721718e+09,"<a href=""http://www.rfs.nsw.gov.au"" rel=""nofol...",ADVICE. Grass Fire. Lake Macquarie (8 Border S...,NaN,NaN,NaN,...,Eraring,Australia,"Eraring, Morisset - Cooranbong",neighborhood,AU,2017-05-14,Lake Macquarie,NSW,-33.065000,151.518000
4,6,44,7.922930e+17,2016-10-29 09:12:34.000000,4.721718e+09,"<a href=""http://www.rfs.nsw.gov.au"" rel=""nofol...","ADVICE. Grass Fire. Temora (Goldfields Way, Te...",NaN,NaN,NaN,...,Temora,Australia,"Temora, New South Wales",neighborhood,AU,2017-05-14,Temora,NSW,-34.445032,147.536098


In [10]:
lga_tweets_df = tweetdf[['lga','tweet_id']].groupby(['lga']).count().reset_index()
lga_tweets_df.columns = ['lga','total_tweets']
lga_tweets_df

,lga,total_tweets
0,Albury,20875
1,Armidale Regional,5790
2,Ballina,5462
3,Balranald,88
4,Bathurst Regional,6114
5,Bayside,4984
6,Bega Valley,4677
7,Bellingen,1523
8,Berrigan,1109
9,Blacktown,2671


In [12]:
lga_tweeters_df = tweetdf[['lga','from_user_id']].groupby(['lga']).agg({'from_user_id': lambda x: x.nunique()}).reset_index()
lga_tweeters_df.columns = ['lga','total_tweeters']
lga_tweeters_df


,lga,total_tweeters
0,Albury,766.0
1,Armidale Regional,279.0
2,Ballina,698.0
3,Balranald,23.0
4,Bathurst Regional,598.0
5,Bayside,759.0
6,Bega Valley,476.0
7,Bellingen,196.0
8,Berrigan,85.0
9,Blacktown,347.0


In [31]:
derived_df = pd.merge(lga_tweets_df, lga_tweeters_df, on='lga')
derived_df

,lga,total_tweets,total_tweeters
0,Albury,20875,766.0
1,Armidale Regional,5790,279.0
2,Ballina,5462,698.0
3,Balranald,88,23.0
4,Bathurst Regional,6114,598.0
5,Bayside,4984,759.0
6,Bega Valley,4677,476.0
7,Bellingen,1523,196.0
8,Berrigan,1109,85.0
9,Blacktown,2671,347.0


In [14]:
demographics_df = pd.read_csv('../../data/clean/2016_demographics.csv')
demographics_df.rename(columns={'Tot_P_P_G01': 'population'}, inplace=True)
demographics_df

,Unnamed: 0,lga,OPDs_Separate_house_Dwellings_G32,OPDs_Separate_house_Persons_G32,OPDs_SD_r_t_h_th_1_sty_Dwgs_G32,OPDs_SD_r_t_h_th_1_sty_Psns_G32,OPDs_SD_r_t_h_th_2_m_sty_Dwgs_G32,OPDs_SD_r_t_h_th_2_m_sty_Psns_G32,OPDs_SD_r_t_h_th_Tot_Dwgs_G32,OPDs_SD_r_t_h_th_Tot_Psns_G32,...,Part_inc_stated_Fam_household_G29,Part_inc_std_Non_fam_hhds_G29,Partial_income_stated_Tot_G29,All_incomes_ns_Famly_househld_G29,All_inc_ns_Non_famly_househld_G29,All_incomes_not_stated_Tot_G29,Tot_Family_households_G29,Tot_Non_family_households_G29,Tot_Tot_G29,area
0,0,Albury,15634,39601,2907,4658,416,675,3324,5334,...,1136,144,1280,236,320,554,12674,6824,19495,305.9459
1,1,Armidale Regional,8689,22010,771,1404,144,240,910,1637,...,680,89,765,116,186,299,6803,3469,10276,8620.6990
2,2,Ballina,11223,29084,2129,3930,1486,2824,3612,6757,...,994,119,1118,179,311,488,11067,5156,16220,484.9389
3,3,Balranald,734,1907,15,23,3,3,22,24,...,83,6,86,18,24,39,563,240,807,21690.6753
4,4,Bathurst Regional,12166,32367,1137,1974,327,583,1460,2559,...,942,108,1054,164,214,377,9934,4478,14411,3817.8646
5,5,Bayside,21535,67356,4479,10788,3150,8947,7632,19731,...,3656,580,4231,467,615,1078,38482,15896,54377,49.8904
6,6,Bega Valley,10934,25770,834,1296,499,845,1336,2140,...,976,52,1033,144,257,399,8873,4344,13222,6278.8811
7,7,Bellingen,4466,10684,184,339,53,112,233,449,...,368,21,388,48,103,151,3370,1603,4970,1600.4337
8,8,Berrigan,2904,6728,107,139,0,0,107,139,...,199,12,216,51,97,155,2103,1159,3265,2065.7759
9,9,Blacktown,78774,257809,6231,18356,10055,32142,16286,50499,...,7193,390,7586,1163,1005,2168,83375,17730,101107,240.0487


In [32]:
derived_df = pd.merge(demographics_df[['lga','population']], derived_df,on='lga')
derived_df['tweets_per_capita'] = derived_df['total_tweets'] / derived_df['population']
derived_df['tweeters_per_capita'] = derived_df['total_tweeters'] / derived_df['population']
derived_df

,lga,population,total_tweets,total_tweeters,tweets_per_capita,tweeters_per_capita
0,Albury,51076,20875,766.0,0.408705,0.014997
1,Armidale Regional,29449,5790,279.0,0.196611,0.009474
2,Ballina,41790,5462,698.0,0.130701,0.016703
3,Balranald,2287,88,23.0,0.038478,0.010057
4,Bathurst Regional,41300,6114,598.0,0.148039,0.014479
5,Bayside,156058,4984,759.0,0.031937,0.004864
6,Bega Valley,33253,4677,476.0,0.140649,0.014314
7,Bellingen,12668,1523,196.0,0.120224,0.015472
8,Berrigan,8462,1109,85.0,0.131056,0.010045
9,Blacktown,336962,2671,347.0,0.007927,0.001030


In [33]:
# Language booleans
langdf = tweetdf.lang.str.get_dummies()
langs = langdf.columns.tolist()
langdf = pd.concat([tweetdf,langdf],axis=1)[['lga']+langs]
print(langdf)

                            lga  ar  bg  bn  bo  cs  cy  da  de  el ...  sl  \
0                        Dungog   0   0   0   0   0   0   0   0   0 ...   0   
1       Port Macquarie-Hastings   0   0   0   0   0   0   0   0   0 ...   0   
2                       Lithgow   0   0   0   0   0   0   0   0   0 ...   0   
3                Lake Macquarie   0   0   0   0   0   0   0   0   0 ...   0   
4                        Temora   0   0   0   0   0   0   0   0   0 ...   0   
5                         Cowra   0   0   0   0   0   0   0   0   0 ...   0   
6                      Nambucca   0   0   0   0   0   0   0   0   0 ...   0   
7                       Lachlan   0   0   0   0   0   0   0   0   0 ...   0   
8               Clarence Valley   0   0   0   0   0   0   0   0   0 ...   0   
9                 Central Coast   0   0   0   0   0   0   0   0   0 ...   0   
10                       Orange   0   0   0   0   0   0   0   0   0 ...   0   
11                  Broken Hill   0   0   0   0   0 

In [34]:
DROP_COUNT=len(tweetdf) / 500
lga_langdf = langdf.groupby(['lga']).sum().reset_index()
sums = langdf.drop('lga',axis=1).sum().to_dict()
to_drop = [k for (k,v) in sums.items() if k == 'en' or v < DROP_COUNT]
lga_langdf = lga_langdf.drop(to_drop, axis=1)
lga_langdf

,lga,es,fr,in,tl,tr,und
0,Albury,28,39,36,66,4,1283
1,Armidale Regional,11,11,12,52,0,185
2,Ballina,7,5,11,14,0,176
3,Balranald,0,0,0,0,0,3
4,Bathurst Regional,13,14,7,12,0,166
5,Bayside,13,13,12,44,10,275
6,Bega Valley,27,5,25,52,1,344
7,Bellingen,0,3,4,20,0,29
8,Berrigan,11,5,13,8,1,91
9,Blacktown,4,3,26,33,2,163


Language counts will probably not be useful unless converted to per_capita or ratio. As such we would probably need to choose specific languages of interest rather than dynmically choosing them as in the code above. Furthermore the distribution across LGAs does not seem likely to be predictive, with the possibel exception of "und" (undetermined) and tr (Turkish). Unique tweeters per language may be of marginally more interest. Given there is also a risk this will be considered to be a form of ethnic profiling it is probably safer to leave these features out.
It is somewhat surprising that the most common languages apart from english and undetermined are es (spanish), fr (french), in (indonesian/malay), tl (tagalog/phillipines) and tr (turkish).

In [35]:
replies_df = tweetdf[tweetdf.to_user_id.notnull()]
lga_replies_df = replies_df[['lga','to_user_id']].groupby(['lga']).agg({'to_user_id': ['count']}).reset_index()
lga_replies_df.columns = ['lga','total_reply_tweets']
lga_replies_df

,lga,total_reply_tweets
0,Albury,8079
1,Armidale Regional,2230
2,Ballina,1934
3,Balranald,45
4,Bathurst Regional,2269
5,Bayside,2612
6,Bega Valley,1792
7,Bellingen,170
8,Berrigan,556
9,Blacktown,1357


In [36]:
lga_replyers_df = replies_df[['lga','from_user_id']].groupby(['lga']).agg({'from_user_id': lambda x: x.nunique()}).reset_index()
lga_replyers_df.columns = ['lga','total_replyers']
lga_replyers_df

,lga,total_replyers
0,Albury,322.0
1,Armidale Regional,97.0
2,Ballina,161.0
3,Balranald,3.0
4,Bathurst Regional,180.0
5,Bayside,239.0
6,Bega Valley,150.0
7,Bellingen,41.0
8,Berrigan,26.0
9,Blacktown,87.0


In [47]:
derived_df = pd.merge(derived_df, lga_langdf[['lga','und']],on='lga')
derived_df['und_ratio'] = derived_df['und'] / derived_df['total_tweets']
derived_df = pd.merge(derived_df, lga_replies_df[['lga','total_reply_tweets']],on='lga')
derived_df['reply_ratio'] = derived_df['total_reply_tweets'] / derived_df['total_tweets']
derived_df['replies_per_capita'] = derived_df['total_reply_tweets'] / derived_df['population']
derived_df = pd.merge(derived_df, lga_replyers_df[['lga','total_replyers']],on='lga')
derived_df['replyer_ratio'] = derived_df['total_replyers'] / derived_df['total_tweeters']
derived_df['replyers_per_capita'] = derived_df['total_replyers'] / derived_df['population']
derived_df

,lga,population,total_tweets,total_tweeters,tweets_per_capita,tweeters_per_capita,und,und_ratio,total_reply_tweets,reply_ratio,replies_per_capita,total_replyers,replyer_ratio,replyers_per_capita
0,Albury,51076,20875,766.0,0.408705,0.014997,1283,0.061461,8079,0.387018,0.158176,322.0,0.420366,0.006304
1,Armidale Regional,29449,5790,279.0,0.196611,0.009474,185,0.031952,2230,0.385147,0.075724,97.0,0.347670,0.003294
2,Ballina,41790,5462,698.0,0.130701,0.016703,176,0.032223,1934,0.354083,0.046279,161.0,0.230659,0.003853
3,Balranald,2287,88,23.0,0.038478,0.010057,3,0.034091,45,0.511364,0.019676,3.0,0.130435,0.001312
4,Bathurst Regional,41300,6114,598.0,0.148039,0.014479,166,0.027151,2269,0.371115,0.054939,180.0,0.301003,0.004358
5,Bayside,156058,4984,759.0,0.031937,0.004864,275,0.055177,2612,0.524077,0.016737,239.0,0.314888,0.001531
6,Bega Valley,33253,4677,476.0,0.140649,0.014314,344,0.073551,1792,0.383152,0.053890,150.0,0.315126,0.004511
7,Bellingen,12668,1523,196.0,0.120224,0.015472,29,0.019041,170,0.111622,0.013420,41.0,0.209184,0.003237
8,Berrigan,8462,1109,85.0,0.131056,0.010045,91,0.082056,556,0.501353,0.065706,26.0,0.305882,0.003073
9,Blacktown,336962,2671,347.0,0.007927,0.001030,163,0.061026,1357,0.508049,0.004027,87.0,0.250720,0.000258


In [48]:
# source -> contains "instagram.com" becomes is_instagram
instadf = tweetdf[tweetdf.source.str.contains('http://instagram\.com')]
lga_instadf = instadf[['lga','source']].groupby(['lga']).agg({'source': ['count']}).reset_index()
lga_instadf.columns = ['lga','total_instagrams']
lga_instadf

,lga,total_instagrams
0,Albury,1498
1,Armidale Regional,494
2,Ballina,1839
3,Balranald,13
4,Bathurst Regional,1272
5,Bayside,107
6,Bega Valley,870
7,Bellingen,627
8,Berrigan,79
9,Blacktown,38


In [50]:
lga_instagrammers_df = instadf[['lga','from_user_id']].groupby(['lga']).agg({'from_user_id': lambda x: x.nunique()}).reset_index()
lga_instagrammers_df.columns = ['lga','total_instagrammers']
lga_instagrammers_df

,lga,total_instagrammers
0,Albury,256.0
1,Armidale Regional,92.0
2,Ballina,415.0
3,Balranald,11.0
4,Bathurst Regional,284.0
5,Bayside,33.0
6,Bega Valley,228.0
7,Bellingen,109.0
8,Berrigan,32.0
9,Blacktown,21.0


In [52]:
linkdf = tweetdf[tweetdf.text.str.contains('http')]
lga_linkdf = linkdf[['lga','tweet_id']].groupby(['lga']).count().reset_index()
lga_linkdf.columns = ['lga','total_link_tweets']
lga_linkdf

,lga,total_link_tweets
0,Albury,6555
1,Armidale Regional,2883
2,Ballina,3007
3,Balranald,36
4,Bathurst Regional,2687
5,Bayside,1320
6,Bega Valley,2835
7,Bellingen,1276
8,Berrigan,344
9,Blacktown,1016


In [53]:
derived_df = pd.merge(derived_df, lga_instadf,on='lga')
derived_df = pd.merge(derived_df, lga_instagrammers_df,on='lga')
derived_df = pd.merge(derived_df, lga_linkdf,on='lga')
derived_df['instagram_ratio'] = derived_df['total_instagrams'] / derived_df['total_tweets']
derived_df['instagrammers_ratio'] = derived_df['total_instagrammers'] / derived_df['total_tweeters']
derived_df['link_tweets_ratio'] = derived_df['total_link_tweets'] / derived_df['total_tweets']
derived_df

,lga,population,total_tweets,total_tweeters,tweets_per_capita,tweeters_per_capita,und,und_ratio,total_reply_tweets,reply_ratio,replies_per_capita,total_replyers,replyer_ratio,replyers_per_capita,total_instagrams,total_instagrammers,total_link_tweets,instagram_ratio,instagrammers_ratio,link_tweets_ratio
0,Albury,51076,20875,766.0,0.408705,0.014997,1283,0.061461,8079,0.387018,0.158176,322.0,0.420366,0.006304,1498,256.0,6555,0.071760,0.334204,0.314012
1,Armidale Regional,29449,5790,279.0,0.196611,0.009474,185,0.031952,2230,0.385147,0.075724,97.0,0.347670,0.003294,494,92.0,2883,0.085320,0.329749,0.497927
2,Ballina,41790,5462,698.0,0.130701,0.016703,176,0.032223,1934,0.354083,0.046279,161.0,0.230659,0.003853,1839,415.0,3007,0.336690,0.594556,0.550531
3,Balranald,2287,88,23.0,0.038478,0.010057,3,0.034091,45,0.511364,0.019676,3.0,0.130435,0.001312,13,11.0,36,0.147727,0.478261,0.409091
4,Bathurst Regional,41300,6114,598.0,0.148039,0.014479,166,0.027151,2269,0.371115,0.054939,180.0,0.301003,0.004358,1272,284.0,2687,0.208047,0.474916,0.439483
5,Bayside,156058,4984,759.0,0.031937,0.004864,275,0.055177,2612,0.524077,0.016737,239.0,0.314888,0.001531,107,33.0,1320,0.021469,0.043478,0.264848
6,Bega Valley,33253,4677,476.0,0.140649,0.014314,344,0.073551,1792,0.383152,0.053890,150.0,0.315126,0.004511,870,228.0,2835,0.186017,0.478992,0.606158
7,Bellingen,12668,1523,196.0,0.120224,0.015472,29,0.019041,170,0.111622,0.013420,41.0,0.209184,0.003237,627,109.0,1276,0.411687,0.556122,0.837820
8,Berrigan,8462,1109,85.0,0.131056,0.010045,91,0.082056,556,0.501353,0.065706,26.0,0.305882,0.003073,79,32.0,344,0.071235,0.376471,0.310189
9,Blacktown,336962,2671,347.0,0.007927,0.001030,163,0.061026,1357,0.508049,0.004027,87.0,0.250720,0.000258,38,21.0,1016,0.014227,0.060519,0.380382


In [54]:
derived_df.to_csv("../../data/clean/derived_features_2016.csv",index=False)

In [116]:
cols = ['a','b','c']
cols.remove('b')
cols[:0] = 'b'
cols

['b', 'a', 'c']